In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Create the CNN model
model = Sequential()

# Add the first convolutional layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((3, 3)))

# Add the second convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((3, 3)))

# Add the third convolutional layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Flatten the output
model.add(Flatten())

# Add the first dense layer
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Add the output layer
model.add(Dense(4, activation='softmax'))



In [3]:
####rank detection datasets defined
train_path = "cards_dataset_by_suit\\train"
valid_path = "cards_dataset_by_suit\\valid"
test_path = "cards_dataset_by_suit\\test"

batch_size = 32
img_height = 224
img_width = 224
num_classes = 4  # Number of classes in your dataset

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'  # Use one-hot encoded labels
)

# Load validation dataset with integer-encoded labels
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_path,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical' 
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',  # Assuming integer-encoded labels for testing dataset
    shuffle=False  # Ensure dataset is not shuffled
)


Found 7509 files belonging to 4 classes.
Found 260 files belonging to 4 classes.
Found 260 files belonging to 4 classes.


In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:

hist = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
235/235 [==============================] - 212s 897ms/step - loss: 0.9725 - accuracy: 0.5968 - val_loss: 0.8291 - val_accuracy: 0.6154
Epoch 2/10
235/235 [==============================] - 295s 1s/step - loss: 0.3854 - accuracy: 0.8518 - val_loss: 0.2887 - val_accuracy: 0.9000
Epoch 3/10
235/235 [==============================] - 722s 3s/step - loss: 0.2688 - accuracy: 0.9012 - val_loss: 0.2230 - val_accuracy: 0.9231
Epoch 4/10
235/235 [==============================] - 288s 1s/step - loss: 0.2153 - accuracy: 0.9224 - val_loss: 0.1751 - val_accuracy: 0.9462
Epoch 5/10
235/235 [==============================] - 304s 1s/step - loss: 0.1531 - accuracy: 0.9430 - val_loss: 0.2298 - val_accuracy: 0.9115
Epoch 6/10
235/235 [==============================] - 303s 1s/step - loss: 0.1224 - accuracy: 0.9588 - val_loss: 0.4261 - val_accuracy: 0.8077
Epoch 7/10
235/235 [==============================] - 303s 1s/step - loss: 0.0934 - accuracy: 0.9692 - val_loss: 0.4128 - val_accuracy: 0.8

In [6]:
addhist = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2
)

Epoch 1/2
235/235 [==============================] - 221s 937ms/step - loss: 0.0569 - accuracy: 0.9823 - val_loss: 0.1426 - val_accuracy: 0.9654
Epoch 2/2
235/235 [==============================] - 212s 902ms/step - loss: 0.0476 - accuracy: 0.9855 - val_loss: 0.1703 - val_accuracy: 0.9423


In [7]:
model.save('suit_classification')
model.save_weights('suit_classification.h5')

INFO:tensorflow:Assets written to: suit_classification\assets


INFO:tensorflow:Assets written to: suit_classification\assets


In [8]:
rank_model = tf.keras.models.load_model('suit_classification')

In [9]:
test_loss, test_accuracy = rank_model.evaluate(test_ds)

print(f'Test accuracy: {test_accuracy}')

9/9 [==============================] - 2s 188ms/step - loss: 0.1593 - accuracy: 0.9423
Test accuracy: 0.942307710647583


In [10]:
def preprocess_image(image_path, desired_size=(224, 224)):
    # Open the image
    image = Image.open(image_path)
    
    # Resize the image using the ANTIALIAS (high-quality) interpolation method
    # resized_image = image.resize(desired_size, Image.ANTIALIAS)
    
    # Return the resized image
    return image
    # return resized_image


In [32]:
from PIL import Image
import tensorflow as tf
classes = ["clubs", "diamonds", "hearts", "spades"]
# File path to the image
# image_path = "C:\\Users\\jason\\ECE_479\\final_project_3\\cards_dataset_by_suit\\test\\clubs\\1_1.jpg"
image_path = "cropped_image.jpg"
# Load and preprocess the image using TensorFlow
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
image_array = tf.keras.preprocessing.image.img_to_array(image)
image_array = tf.expand_dims(image_array, axis=0)  # Add batch dimension

# Make prediction
predictions = model.predict(image_array)
predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", classes[int(predicted_class)])

1/1 [==============================] - 0s 96ms/step
Predicted class: diamonds
